In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000)
os.environ['USE_PYGEOS'] = '0'
import shared_utils

from calitp_data_analysis.tables import tbls
import calitp_data_analysis.magics
from calitp_data.storage import get_fs

from siuba import *
import pandas as pd
import geopandas as gpd

import datetime as dt

In [2]:
import segment_speed_utils

In [3]:
analysis_date = dt.date(2023, 4, 12)

In [ ]:
fs = get_fs()

In [ ]:
# path = 'gs://calitp-analytics-data/data-analyses/rt_segment_speeds/export/avg_speeds_stop_segments_2023-04-12.parquet'

In [ ]:
# fs.get(path, './avg_speeds_stop_segments_2023-04-12.parquet')

In [ ]:
exported_segments = pd.read_parquet('./avg_speeds_stop_segments_2023-04-12.parquet')

In [ ]:
exported_segments >> head(1)

In [ ]:
gdf = gpd.GeoDataFrame(exported_segments,
                 geometry=gpd.GeoSeries.from_wkb(exported_segments.geometry),
                 crs=segment_speed_utils.project_vars.PROJECT_CRS
                )

In [ ]:
gdf >> head(3)

In [ ]:
# opening direct from gcs seemed to hang?
# gpd.read_file('./avg_speeds_stop_segments_2023-04-12.parquet')

In [ ]:
shared_utils.rt_utils.get_speedmaps_ix_df?

In [ ]:
shared_utils.rt_utils.get_speedmaps_ix_df(analysis_date) >> filter(_.organization_name.str.contains('Santa')) >> head(5)

In [ ]:
samo_url = (tbls.mart_transit_database.dim_gtfs_datasets()
          >> filter(_.key == '6c2d7daaf979779fa2089c6395baf98b') # Santa Monica
          >> collect()
         ).base64_url.iloc[0]

In [ ]:
segments_samo = (gdf >> filter(_.time_of_day == 'all_day', _.base64_url == samo_url)).copy()

In [ ]:
segments_samo >> head(3)

In [ ]:
type(ZERO_THIRTY_COLORSCALE)

In [ ]:
import branca

In [ ]:
isinstance(ZERO_THIRTY_COLORSCALE, branca.colormap.ColorMap)

In [ ]:
from shared_utils.rt_utils import try_parallel, arrowize_by_frequency, ZERO_THIRTY_COLORSCALE

In [ ]:
segments_samo.geometry = segments_samo.geometry.apply(try_parallel)

In [ ]:
segments_samo = segments_samo.apply(arrowize_by_frequency, axis=1, args=['n_trips'])

In [ ]:
segments_samo.explore()

In [ ]:
## TODO this on an SPA map... 

## Segs to spa util

In [ ]:
SPA_MAP_SITE = 'https://embeddable-maps.calitp.org/'
SPA_MAP_BUCKET = 'calitp-map-tiles/'
SPEEDMAP_LEGEND_URL = 'https://storage.googleapis.com/calitp-map-tiles/speeds_legend.svg'

In [ ]:
import gzip
import base64
import json
from calitp_data.storage import get_fs

In [ ]:
def spa_map_export_link(gdf: gpd.GeoDataFrame, path: str, state: dict, site: str=SPA_MAP_SITE):
## TODO generalize and --> shared_utils
    fs = get_fs()

    geojson_str = gdf.to_json()
    geojson_bytes = geojson_str.encode('utf-8')
    print(f'writing to {path}')
    with fs.open(path, 'wb') as writer:  # write out to public-facing GCS?
        with gzip.GzipFile(fileobj=writer, mode="w") as gz:
            gz.write(geojson_bytes)
    base64state = base64.urlsafe_b64encode(json.dumps(state).encode()).decode()
    spa_map_url = f'{site}?state={base64state}'
    return spa_map_url

In [ ]:
if 1 == 1: print('yeah') 

In [ ]:
{} or {2:'2'}

In [ ]:
def set_state_export(gdf, bucket: str=SPA_MAP_BUCKET, subfolder: str='testing/',
                     filename: str='test2', map_type: str=None, map_title: str='Map',
                    cbar: branca.colormap.ColorMap=None, color_col: str=None,
                    legend_url: str=None, existing_layers: list=[]):
    '''
    
    '''
    
    spa_map_state = {"name": "null", "layers": existing_layers, "lat_lon": (),
                             "zoom": 13}
    path = f'{bucket}{subfolder}{filename}.geojson.gz'
    gdf = gdf.to_crs(shared_utils.geography_utils.WGS84)
    if cbar and color_col:
        gdf['color'] = gdf[color_col].apply(lambda x: cbar.rgb_bytes_tuple(x))
    gdf = gdf.round(2) # round for map display
    this_layer = [{
        "name": f"{map_title}",
        "url": f"https://storage.googleapis.com/{path}",
        'properties': {'stroked': False, 'highlight_saturation_multiplier': 0.5}
        }]
    if map_type: this_layer[0]['type'] = map_type 
    if map_type == 'speedmap':
        this_layer[0]['properties']['tooltip_speed_key'] = 'p20_mph'
    spa_map_state["layers"] += this_layer
    centroid = (gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean())
    spa_map_state["lat_lon"] = centroid
    if legend_url:
        spa_map_state['legend_url'] = legend_url
    return {'state_dict': spa_map_state, 'spa_link': spa_map_export_link(gdf = gdf, path = path, state = spa_map_state)}

In [ ]:
set_state_export(segments_samo, cbar=ZERO_THIRTY_COLORSCALE, legend_url=SPEEDMAP_LEGEND_URL, map_type='speedmap')

In [ ]:
set_state_export(segments_samo)

## Speedmap Comparision

In [4]:
from rt_analysis import rt_filter_map_plot

In [5]:
rt_day = rt_filter_map_plot.from_gcs(300, analysis_date)

found shapes parquet at gs://calitp-analytics-data/data-analyses/rt_delay/v2_cached_views/shapes_300_2023-04-12.parquet


In [6]:
_m = rt_day.segment_speed_map(no_render=True)

/home/jovyan/data-analyses/rt_delay/rt_analysis/rt_filter_map_plot.py:449: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

/home/jovyan/data-analyses/rt_delay/rt_analysis/rt_filter_map_plot.py:449: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.



In [7]:
rt_day.map_gz_export()

AttributeError: 'RtFilterMapper' object has no attribute 'spa_map_state'

In [ ]:
rt_day.spa_map_state

In [ ]:
rt_day.spa_map_url

In [ ]:
## TODO shn?

In [ ]:
rt_day.display_spa_map()